In [1]:
# Import library untuk menghubungkan Google Drive dengan Google Colab
from google.colab import drive

# Mount Google Drive pada Google Colab
drive.mount('/content/drive')

# Set path ke folder yang berisi file JSON
import os
try:
  os.chdir('/content/drive/MyDrive/Magang/Penelitian_AI') #Ahmad
except:
  try:
    os.chdir('/content/drive/MyDrive/Semester6/Magang/Penelitian_AI') #Thoriq
  except:
    os.chdir('/content/drive/MyDrive/JSON') #Sultan


Mounted at /content/drive


In [2]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# set kredensial untuk API access
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'key_GoogleSheetAPI.json'
creds = None
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [3]:
SPREADSHEET_ID = '1UeT7sqIsS8Ni3yLz6fh5GOBNJveyCpF_XAtQMRWC1Mo'
RANGE_NAME = 'Sheet1!J2:L1216'

# buat koneksi ke Google Sheets API
service = build('sheets', 'v4', credentials=creds)
result = service.spreadsheets().values().get(
    spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])

print(values)

dataSuhuInd, dataSuhuOut, dataSuhu_ac = [], [], []

for sublist in values:
  x1, x2, x3 = sublist
  dataSuhuInd.append(float(x1))
  dataSuhuOut.append(float(x2))
  dataSuhu_ac.append(float(x3))

print(dataSuhuInd)
print(dataSuhuOut)
print(dataSuhu_ac)
print(len(dataSuhuInd), len(dataSuhuOut), len(dataSuhu_ac))

[['19', '29', '20'], ['19.2', '29', '20'], ['18.7', '29', '20'], ['19.6', '29', '20'], ['18.7', '29', '20'], ['19.4', '29', '20'], ['20.4', '29', '20'], ['20.4', '29', '20'], ['20.5', '29', '20'], ['20.6', '29', '20'], ['20.9', '29', '20'], ['21.1', '29', '20'], ['21', '29', '20'], ['21', '29', '20'], ['20.9', '29', '20'], ['20.9', '29', '20'], ['21', '29', '20'], ['21.1', '29', '20'], ['21.1', '29', '20'], ['21.4', '29', '20'], ['21.3', '29', '20'], ['21.4', '29', '20'], ['21.6', '29', '20'], ['21.8', '29', '20'], ['21.8', '29', '20'], ['21.8', '29', '20'], ['21.8', '29', '20'], ['21.9', '29', '20'], ['22', '29', '20'], ['22', '29', '20'], ['21.8', '29', '20'], ['22.1', '29', '21'], ['22.2', '29', '21'], ['22.2', '29', '21'], ['22.1', '29', '21'], ['22.4', '29', '21'], ['22.4', '29', '21'], ['22.4', '29', '21'], ['22.6', '29', '21'], ['22.7', '29', '21'], ['23.1', '29', '21'], ['23.4', '29', '21'], ['23.2', '29', '21'], ['23.3', '29', '21'], ['23.5', '29', '21'], ['23.5', '29', '21'],

In [5]:
import numpy as np

num_samples = 1215
indTemp = np.array(dataSuhuInd)
outTemp = np.array(dataSuhuOut)
ac_temp = np.array(dataSuhu_ac)

# Normalize the data Z-Score
indTemp_norm = (indTemp - np.mean(indTemp)) / np.std(indTemp)
outTemp_norm = (outTemp - np.mean(outTemp)) / np.std(outTemp)
ac_temp_norm = (ac_temp - np.mean(ac_temp)) / np.std(ac_temp)

In [6]:
# # Import library yang dibutuhkan
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_absolute_error

# # Konversi data ke dalam format dataframe pandas
# data = pd.DataFrame({'suhu_dalam_ruangan': [float(i[0]) for i in values1],
#                      'suhu_luar_ruangan': [float(i[0]) for i in values2],
#                      'suhu_AC' : [float(i) for i in range(1000)]})

# # Pisahkan variabel input dan target
# X = data[['suhu_dalam_ruangan', 'suhu_luar_ruangan', 'suhu_AC']]
# y = data['suhu_AC']

# # Bagi data menjadi data latih dan data uji
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Hitung nilai rata-rata dari target pada data latih
# mean = np.mean(y_train)

# # Lakukan prediksi menggunakan nilai rata-rata
# y_pred = np.full_like(y_test, mean)

# # Hitung MAE
# mae = mean_absolute_error(y_test, y_pred)

# # Cetak hasil
# print('MAE: %.2f' % mae)

In [7]:
# Create the input data and output labels
X = np.column_stack((outTemp_norm, ac_temp_norm))
y = indTemp_norm.reshape(-1, 1)

In [8]:
# Split the data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * num_samples)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

In [9]:
import tensorflow as tf
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=1000, batch_size=32,
                    validation_data=(X_test, y_test))

Epoch 1/1000
31/31 [==============================] - 2s 14ms/step - loss: 0.7866 - mae: 0.7218 - val_loss: 0.4880 - val_mae: 0.5888
Epoch 2/1000
31/31 [==============================] - 0s 4ms/step - loss: 0.6595 - mae: 0.6511 - val_loss: 0.4670 - val_mae: 0.5716
Epoch 3/1000
31/31 [==============================] - 0s 4ms/step - loss: 0.6172 - mae: 0.6327 - val_loss: 0.4800 - val_mae: 0.5762
Epoch 4/1000
31/31 [==============================] - 0s 4ms/step - loss: 0.5830 - mae: 0.6142 - val_loss: 0.5117 - val_mae: 0.5830
Epoch 5/1000
31/31 [==============================] - 0s 4ms/step - loss: 0.5475 - mae: 0.5937 - val_loss: 0.5035 - val_mae: 0.5639
Epoch 6/1000
31/31 [==============================] - 0s 5ms/step - loss: 0.5317 - mae: 0.5820 - val_loss: 0.5590 - val_mae: 0.5698
Epoch 7/1000
31/31 [==============================] - 0s 4ms/step - loss: 0.5199 - mae: 0.5720 - val_loss: 0.6138 - val_mae: 0.5878
Epoch 8/1000
31/31 [==============================] - 0s 5ms/step - loss: 0

In [12]:
# Evaluate model
print(model.evaluate(X_test, y_test))

8/8 [==============================] - 0s 2ms/step - loss: 0.5457 - mae: 0.5417
[0.5456975698471069, 0.5416533946990967]


In [18]:
# Mengambil data suhu luar ruangan dan suhu AC besok
suhu_luar_besok = 30.0
suhu_ac_besok = 16.0

# Normalisasi z-score pada data suhu luar ruangan dan suhu AC besok
suhu_luar_besok_norm = (suhu_luar_besok - np.mean(dataSuhuOut)) / np.std(dataSuhuOut)
suhu_ac_besok_norm = (suhu_ac_besok - np.mean(dataSuhu_ac)) / np.std(dataSuhu_ac)

print(np.mean(dataSuhuOut))
print(np.mean(dataSuhuInd))
print(np.mean(dataSuhu_ac))
print(np.std(dataSuhuOut))
print(np.std(dataSuhuInd))
print(np.std(dataSuhu_ac))

# Membuat input data untuk prediksi
input_data = np.array([[suhu_luar_besok_norm, suhu_ac_besok_norm]])

# Melakukan prediksi suhu dalam ruangan besok
prediksi_suhu_besok = model.predict(input_data)

# Mengembalikan prediksi suhu dalam skala asli
prediksi_suhu_besok = prediksi_suhu_besok * np.std(dataSuhuInd) + np.mean(dataSuhuInd)

print('Prediksi suhu dalam ruangan besok:', prediksi_suhu_besok[0][0])

25.2320987654321
22.713827160493828
20.776954732510287
2.3037897116518424
1.5215510019739682
1.3482360774673696
1/1 [==============================] - 0s 32ms/step
Prediksi suhu dalam ruangan besok: 21.363173


In [14]:
model.save('mymodel.h5')

In [17]:
# salin model ke Google Drive
!cp mymodel.h5 "/content/drive/MyDrive/Magang/Penelitian_AI" #Toriq
!cp mymodel.h5 "/content/drive/MyDrive/Semester6/Magang/Penelitian_AI" #Ahmad
!cp mymodel.h5 "/content/drive/MyDrive/JSON" #Sultan

cp: cannot create regular file '/content/drive/MyDrive/Magang/Penelitian_AI': No such file or directory
cp: 'mymodel.h5' and '/content/drive/MyDrive/Semester6/Magang/Penelitian_AI/mymodel.h5' are the same file
